In [1]:
from langchain.tools import tool
from langchain.chat_models import init_chat_model


@tool
def get_weather(location:str)->str:
    """Get the weather of the location"""
    return f"The weather of the {location} is sunny"


model = init_chat_model(
    model="gpt-5-nano",
    temperature=0.5,

)

In [11]:
model_with_tools = model.bind_tools([get_weather], parallel_tool_calls=False)

In [12]:
response = model_with_tools.invoke("What is the weather in Boston and Kerala?")
response

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 134, 'total_tokens': 285, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D2WkBDibnqPm90BYyjOX1y412Ik4S', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bfe1d-5324-7f62-8030-318576a507b0-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_bxaGG1OxX76QosupyZ40jwyH', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 134, 'output_tokens': 151, 'total_tokens': 285, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

In [14]:
for tool_call in response.tool_calls:
    print(tool_call["name"])
    print(tool_call["args"])

get_weather
{'location': 'Boston'}


In [7]:
# Bind (potentially multiple) tools to the model
model_with_tools = model.bind_tools([get_weather])

# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

Currently, it's sunny in Boston. Would you like the temperature, humidity, wind details, or a forecast?
